https://chatgpt.com/g/g-p-67abc21fef088191851e82de2cbad81c-icarus-opatija-presentation-japanese-networks/c/67abc3e8-7304-8013-9150-44d69ae2246b

https://www.icarushrvatska.hr/

https://drive.google.com/file/d/1g5uMoRqVytStIV7KLT-7VW6Xrn0VaAsq/view


Abstract
The presentation, titled “Extraction of Semantic Networks using Contemporary AI Based Methods,” explores modern approaches for automatically extracting and analyzing lexical semantic networks within cultural heritage contexts. Focusing on Japanese examples—illustrated here by the term "Samurai"—the research demonstrates how advanced AI algorithms can identify key lexical elements and reveal their interconnections, such as synonyms, classifications, and relation strengths, from large text corpora.

By leveraging integrated data sources like Wikidata, the approach retrieves relevant metadata, including concise definitions, classification attributes (e.g., instance of, subclass of), and even visual elements such as associated images. The method utilizes SPARQL queries on established knowledge graphs to pinpoint the entity corresponding to the target term and to extract its core attributes without delving into excessive detail.

This dynamic extraction process results in interactive network representations that enrich digital cultural archives and museum collections. Users can engage with the network nodes to access supplementary information about the term, thereby gaining insights into historical narratives and cultural significance. The example of "Samurai" is abstracted to illustrate the process in a simplified manner, emphasizing the potential of these AI-based methods to enhance the interpretation and presentation of cultural heritage.

Overall, this approach provides a scalable and efficient tool for digital preservation and scholarly exploration, opening new avenues for research and public engagement in the realm of cultural heritage.

In [1]:
# Instalirajte SPARQLWrapper ako već nije instaliran
!pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 19.7 MB/s eta 0:00:00


In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Definiramo Wikidata SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Postavite japanski kanđi za pretragu (primjer: "文化" za "culture")
japanese_word = "桜"

# SPARQL upit pronalazi entitete s japanskim labelom i dohvaća proširene informacije:
query = f"""
SELECT ?item ?itemLabel ?definition
       ?instanceOf ?instanceOfLabel
       ?subclassOf ?subclassOfLabel
       ?hasPart ?hasPartLabel
       ?partOf ?partOfLabel
       ?officialWebsite
       ?image
WHERE {{
  ?item rdfs:label "{japanese_word}"@ja.
  OPTIONAL {{
    ?item schema:description ?definition .
    FILTER(LANG(?definition) = "en")
  }}
  OPTIONAL {{ ?item wdt:P31 ?instanceOf. }}
  OPTIONAL {{ ?item wdt:P279 ?subclassOf. }}
  OPTIONAL {{ ?item wdt:P527 ?hasPart. }}
  OPTIONAL {{ ?item wdt:P361 ?partOf. }}
  OPTIONAL {{ ?item wdt:P856 ?officialWebsite. }}
  OPTIONAL {{ ?item wdt:P18 ?image. }}
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,hr,ja". }}
}}
LIMIT 20
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Grupiramo rezultate po entitetu – jedan entitet može imati više redaka za različite opcionalne podatke
data = {}

for result in results["results"]["bindings"]:
    item = result["item"]["value"]
    if item not in data:
        data[item] = {
            "Naziv": result.get("itemLabel", {}).get("value", ""),
            "Definicija": result.get("definition", {}).get("value", "Nema definicije"),
            "Instance of": set(),
            "Subclass of": set(),
            "Has part": set(),
            "Part of": set(),
            "Official Website": set(),
            "Slika": result.get("image", {}).get("value", None)
        }
    if "instanceOfLabel" in result:
        data[item]["Instance of"].add(result["instanceOfLabel"]["value"])
    if "subclassOfLabel" in result:
        data[item]["Subclass of"].add(result["subclassOfLabel"]["value"])
    if "hasPartLabel" in result:
        data[item]["Has part"].add(result["hasPartLabel"]["value"])
    if "partOfLabel" in result:
        data[item]["Part of"].add(result["partOfLabel"]["value"])
    if "officialWebsite" in result:
        data[item]["Official Website"].add(result["officialWebsite"]["value"])

# Ispis rezultata
if data:
    for item, details in data.items():
        print("Entitet:", item)
        print("Naziv:", details["Naziv"])
        print("Definicija:", details["Definicija"])
        if details["Instance of"]:
            print("Instance of:", ", ".join(details["Instance of"]))
        if details["Subclass of"]:
            print("Subclass of:", ", ".join(details["Subclass of"]))
        if details["Has part"]:
            print("Has part:", ", ".join(details["Has part"]))
        if details["Part of"]:
            print("Part of:", ", ".join(details["Part of"]))
        if details["Official Website"]:
            print("Official Website:", ", ".join(details["Official Website"]))
        if details["Slika"]:
            print("Slika:", details["Slika"])
        print("-" * 40)
else:
    print("Nema rezultata za zadani upit.")


Entitet: http://www.wikidata.org/entity/Q6158541
Naziv: Japanese destroyer Sakura
Definicija: 1944 Matsu-class destroyer
Instance of: destroyer escort, destroyer
----------------------------------------
Entitet: http://www.wikidata.org/entity/Q11526656
Naziv: Sakura
Definicija: area in Otaru, Hokkaido, Japan
Instance of: chōchō
Slika: http://commons.wikimedia.org/wiki/Special:FilePath/Sakura%20town%20around%20rotary%20intersection%2C%20Otaru%20city%2C%20Hokkaido%287982383476%29.jpg
----------------------------------------
Entitet: http://www.wikidata.org/entity/Q11537654
Naziv: 桜
Definicija: Nema definicije
Instance of: double act
----------------------------------------
Entitet: http://www.wikidata.org/entity/Q11537655
Naziv: 桜
Definicija: Nema definicije
Instance of: neighborhood in Japan
----------------------------------------
Entitet: http://www.wikidata.org/entity/Q11537657
Naziv: Sakura
Definicija: town located in Setagaya-ku, Tokyo
Instance of: chōchō
--------------------------